In [1]:
import tensorflow as tf
import numpy as np
from custom_layers import *
from worker import Worker

In [2]:
coordinator = tf.train.Coordinator()

hidden_layer_size = 32
output_size = 10
global_model = FullyConnectedModel(
                output_size=output_size,
                hidden_layer_size=hidden_layer_size,
            )
#_ = global_model([global_model.inputs['img'], global_model.inputs['mask']])
#print(global_model.accumulators)
fake_img = np.zeros((1, 28, 28))
fake_mask = np.zeros((1, hidden_layer_size))
print("Init call: ", global_model([fake_img, fake_mask]))
'''
for layer in global_model.layers:
    layer.trainable = False
'''
global_model.summary()
print("weights: ", global_model.get_weights())
print("accumulators: ", global_model.accumulators)
global_model.init_accumulators()
print("accumulators: ", global_model.accumulators)
print("weights: ", global_model.get_weights())
print("trainable: ", global_model.non_trainable_weights)
#global_model.copy_weights(global_model)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Init call:  tf.Tensor([[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]], shape=(1, 10), dtype=float32)
Model: "fully_connected_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
fully_connected_model_flatte multiple                  0         
_________________________________________________________________
fully_connected_model_encode multiple                  25120     
____________________________________________________________

In [3]:
global_model.trainable_weights

[<tf.Variable 'fully_connected_model/fully_connected_model_encoder/kernel:0' shape=(784, 32) dtype=float32, numpy=
 array([[-0.00903054, -0.00967028,  0.04543597, ...,  0.04406559,
          0.01756281, -0.00753082],
        [-0.00216007,  0.08010327,  0.02851767, ...,  0.01676904,
          0.00373536,  0.0753483 ],
        [ 0.00966362, -0.05113121, -0.06015578, ...,  0.00426949,
          0.01668978,  0.01631317],
        ...,
        [ 0.06652216, -0.06601961,  0.01898639, ..., -0.06333527,
         -0.04236922, -0.00488206],
        [ 0.01286289,  0.02781226, -0.0621433 , ..., -0.07216462,
         -0.08376853,  0.04045679],
        [ 0.07076073, -0.01858263, -0.00851363, ...,  0.02329163,
         -0.04008366, -0.05825036]], dtype=float32)>,
 <tf.Variable 'fully_connected_model/fully_connected_model_encoder/bias:0' shape=(32,) dtype=float32, numpy=
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0

In [4]:
num_models = 3
models = {}
for idx in range(num_models):
    model = FullyConnectedModel(
            output_size=output_size,
            hidden_layer_size=hidden_layer_size,
            #parent_model=global_model,
        )
    #_ = model([model.inputs['img'], model.inputs['mask']])
    print(model([fake_img, fake_mask]))
    models[idx] = model
    #model.set_parent_model(global_model)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

tf.Tensor([[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]], shape=(1, 10), dtype=float32)

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting

In [5]:
for idx in range(num_models):
    t = Worker(args=idx, kwargs=
                {
                    'coordinator': coordinator,
                    'global_model': global_model,
                    'model': models[idx],
                    'hidden_layer_size': hidden_layer_size,
                    'output_size': output_size,
                    'learning_rate': 1e-4,
                    'dropout_rate': 1-0.35,
                    'batch_size': 16,
                    'epochs': 10,
                    'update_batches': 100
                }
    )
    t.start()


Model: "fully_connected_model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
fully_connected_model_1_flat multiple                  0         
_________________________________________________________________
fully_connected_model_1_enco multiple                  25120     
_________________________________________________________________
fully_connected_model_1_Cust multiple                  0         
_________________________________________________________________
fully_connected_model_1_outp multiple                  330       
Total params: 25,450
Trainable params: 25,450
Non-trainable params: 0
_________________________________________________________________
None
Model: "fully_connected_model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
fully_connected_model_2_flat multiple                  0         
_

Epoch: 0	Thread: 0	step 1000	mean loss = tf.Tensor(0.08375954, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.35445803, shape=(), dtype=float32)
Epoch: 0	Thread: 2	step 1000	mean loss = tf.Tensor(0.07611643, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.5066184, shape=(), dtype=float32)
Epoch: 0	Thread: 1	step 1100	mean loss = tf.Tensor(0.071077555, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.57958674, shape=(), dtype=float32)
Epoch: 0	Thread: 0	step 1100	mean loss = tf.Tensor(0.0823299, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.3669959, shape=(), dtype=float32)
Epoch: 0	Thread: 2	step 1100	mean loss = tf.Tensor(0.074391186, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.5262829, shape=(), dtype=float32)
Epoch: 0	Thread: 1	step 1200	mean loss = tf.Tensor(0.06911621, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.5981474, shape=(), dtype=float32)
Epoch: 0	Thread: 0	step 1200	mean loss = tf.Tensor(0.08098409, shape=(), dtype=float32)
	me

Epoch: 0	Thread: 2	step 2800	mean loss = tf.Tensor(0.056230076, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.69756335, shape=(), dtype=float32)
Epoch: 0	Thread: 0	step 2800	mean loss = tf.Tensor(0.06780268, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.525839, shape=(), dtype=float32)
Epoch: 0	Thread: 1	step 2900	mean loss = tf.Tensor(0.05224794, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.73849535, shape=(), dtype=float32)
Epoch: 0	Thread: 2	step 2900	mean loss = tf.Tensor(0.055520147, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.7030119, shape=(), dtype=float32)
Epoch: 0	Thread: 0	step 2900	mean loss = tf.Tensor(0.06731078, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.52996814, shape=(), dtype=float32)
Epoch: 0	Thread: 1	step 3000	mean loss = tf.Tensor(0.051587418, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.74285656, shape=(), dtype=float32)
Epoch: 0	Thread: 2	step 3000	mean loss = tf.Tensor(0.05482791, shape=(), dtype=float32)


Epoch: 1	Thread: 2	step 800	mean loss = tf.Tensor(0.04619324, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.7686223, shape=(), dtype=float32)
Epoch: 1	Thread: 0	step 800	mean loss = tf.Tensor(0.060977962, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.5725939, shape=(), dtype=float32)
Epoch: 1	Thread: 1	step 900	mean loss = tf.Tensor(0.04315396, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.79689854, shape=(), dtype=float32)
Epoch: 1	Thread: 2	step 900	mean loss = tf.Tensor(0.04582968, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.7708423, shape=(), dtype=float32)
Epoch: 1	Thread: 0	step 900	mean loss = tf.Tensor(0.060723614, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.57380134, shape=(), dtype=float32)
Epoch: 1	Thread: 1	step 1000	mean loss = tf.Tensor(0.042821083, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.79872656, shape=(), dtype=float32)
Epoch: 1	Thread: 2	step 1000	mean loss = tf.Tensor(0.045441866, shape=(), dtype=float32)
	mea

Epoch: 1	Thread: 1	step 2600	mean loss = tf.Tensor(0.038230427, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.82353175, shape=(), dtype=float32)
Epoch: 1	Thread: 2	step 2700	mean loss = tf.Tensor(0.04033497, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.80452645, shape=(), dtype=float32)
Epoch: 1	Thread: 0	step 2600	mean loss = tf.Tensor(0.057502273, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.59030074, shape=(), dtype=float32)
Epoch: 1	Thread: 1	step 2700	mean loss = tf.Tensor(0.037988123, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.8248624, shape=(), dtype=float32)
Epoch: 1	Thread: 2	step 2800	mean loss = tf.Tensor(0.0400898, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.806022, shape=(), dtype=float32)
Epoch: 1	Thread: 0	step 2700	mean loss = tf.Tensor(0.05735285, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.59099364, shape=(), dtype=float32)
Epoch: 1	Thread: 1	step 2800	mean loss = tf.Tensor(0.03772768, shape=(), dtype=float32)
	

...

Epoch: 6	Thread: 0	step 1600	mean loss = tf.Tensor(0.04606085, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.6405258, shape=(), dtype=float32)
Epoch: 6	Thread: 2	step 1800	mean loss = tf.Tensor(0.024258157, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.89161456, shape=(), dtype=float32)
Epoch: 6	Thread: 1	step 1800	mean loss = tf.Tensor(0.023319745, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.8970979, shape=(), dtype=float32)
Epoch: 6	Thread: 0	step 1700	mean loss = tf.Tensor(0.046013314, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.6409239, shape=(), dtype=float32)
Epoch: 6	Thread: 2	step 1900	mean loss = tf.Tensor(0.024228526, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.89172065, shape=(), dtype=float32)
Epoch: 6	Thread: 1	step 1900	mean loss = tf.Tensor(0.023296326, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.8971763, shape=(), dtype=float32)
Epoch: 6	Thread: 0	step 1800	mean loss = tf.Tensor(0.045958698, shape=(), dtype=float32

Epoch: 6	Thread: 1	step 3500	mean loss = tf.Tensor(0.022874156, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.8989678, shape=(), dtype=float32)
Epoch: 6	Thread: 2	step 3600	mean loss = tf.Tensor(0.023699028, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.8943599, shape=(), dtype=float32)
Epoch: 6	Thread: 0	step 3500	mean loss = tf.Tensor(0.045218624, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.64748955, shape=(), dtype=float32)
Epoch: 6	Thread: 1	step 3600	mean loss = tf.Tensor(0.022836965, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.899161, shape=(), dtype=float32)
Epoch: 6	Thread: 2	step 3700	mean loss = tf.Tensor(0.023651654, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.89463663, shape=(), dtype=float32)
Epoch: 6	Thread: 1	step 3700	mean loss = tf.Tensor(0.022797907, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.8993669, shape=(), dtype=float32)
Epoch: 6	Thread: 0	step 3600	mean loss = tf.Tensor(0.045173027, shape=(), dtype=float32

Epoch: 7	Thread: 1	step 1500	mean loss = tf.Tensor(0.022402601, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.90115, shape=(), dtype=float32)
Epoch: 7	Thread: 2	step 1600	mean loss = tf.Tensor(0.023169737, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.8970392, shape=(), dtype=float32)
Epoch: 7	Thread: 0	step 1500	mean loss = tf.Tensor(0.04446002, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.6535598, shape=(), dtype=float32)
Epoch: 7	Thread: 1	step 1600	mean loss = tf.Tensor(0.022377208, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.9012715, shape=(), dtype=float32)
Epoch: 7	Thread: 2	step 1700	mean loss = tf.Tensor(0.023145385, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.89715934, shape=(), dtype=float32)
Epoch: 7	Thread: 0	step 1600	mean loss = tf.Tensor(0.044420887, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.6538163, shape=(), dtype=float32)
Epoch: 7	Thread: 1	step 1700	mean loss = tf.Tensor(0.022353819, shape=(), dtype=float32)
	

Epoch: 7	Thread: 0	step 3200	mean loss = tf.Tensor(0.043852754, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.6584305, shape=(), dtype=float32)
Epoch: 7	Thread: 1	step 3400	mean loss = tf.Tensor(0.02200761, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.9027942, shape=(), dtype=float32)
Epoch: 7	Thread: 2	step 3400	mean loss = tf.Tensor(0.02273849, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.8991307, shape=(), dtype=float32)
Epoch: 7	Thread: 0	step 3300	mean loss = tf.Tensor(0.043815505, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.65870655, shape=(), dtype=float32)
Epoch: 7	Thread: 1	step 3500	mean loss = tf.Tensor(0.021983055, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.90291923, shape=(), dtype=float32)
Epoch: 7	Thread: 2	step 3500	mean loss = tf.Tensor(0.022712328, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.8992597, shape=(), dtype=float32)
Epoch: 7	Thread: 0	step 3400	mean loss = tf.Tensor(0.043779198, shape=(), dtype=float32)

Epoch: 8	Thread: 0	step 1200	mean loss = tf.Tensor(0.04323197, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.663268, shape=(), dtype=float32)
Epoch: 8	Thread: 2	step 1400	mean loss = tf.Tensor(0.022307115, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.90133077, shape=(), dtype=float32)
Epoch: 8	Thread: 1	step 1400	mean loss = tf.Tensor(0.021628724, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.9045631, shape=(), dtype=float32)
Epoch: 8	Thread: 0	step 1300	mean loss = tf.Tensor(0.043197542, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.6635471, shape=(), dtype=float32)
Epoch: 8	Thread: 2	step 1500	mean loss = tf.Tensor(0.022284871, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.9014337, shape=(), dtype=float32)
Epoch: 8	Thread: 1	step 1500	mean loss = tf.Tensor(0.02161083, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.90463793, shape=(), dtype=float32)
Epoch: 8	Thread: 0	step 1400	mean loss = tf.Tensor(0.04316684, shape=(), dtype=float32)
	